In [ ]:
!pip install -q unsloth "trl>=0.9.0" datasets bitsandbytes accelerate pypdf

import torch, platform, os

print("Python:", platform.python_version())
print("Torch:", torch.__version__)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠️ No se detectó GPU, activa GPU en Runtime ➜ Change runtime type.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.9/373.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048   # puedes subirlo si tu GPU aguanta
MODEL_NAME = "unsloth/Qwen3-4B-Instruct-2507-unsloth-bnb-4bit"  # Qwen3 4B en 4bit

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name       = MODEL_NAME,
    max_seq_length   = max_seq_length,
    load_in_4bit     = True,   # 4-bit QLoRA (ideal para Colab T4)
    load_in_8bit     = False,
    load_in_16bit    = False,
    full_finetuning  = False,  # usamos LoRA (fine-tuning ligero)
)

# Añadir LoRA (parámetros recomendados genéricos)
model = FastLanguageModel.get_peft_model(
    model,
    r                         = 16,
    target_modules            = ["q_proj","k_proj","v_proj","o_proj",
                                 "gate_proj","up_proj","down_proj"],
    lora_alpha                = 16,
    lora_dropout              = 0,
    bias                      = "none",
    use_gradient_checkpointing= "unsloth",
    random_state              = 3407,
    max_seq_length            = max_seq_length,
    use_rslora                = False,
    loftq_config              = None,
)

FastLanguageModel.for_inference(model)
print("✅ Qwen3 cargado con LoRA listo para fine-tuning.")

==((====))==  Unsloth 2025.12.5: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Qwen3 cargado con LoRA listo para fine-tuning.


In [ ]:
def chat_es(prompt, max_new_tokens=256):
    messages = [
        {"role": "system", "content": "Eres un asistente médico clínico en español, preciso y conciso."},
        {"role": "user", "content": prompt},
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.2,
            top_p=0.9,
        )

    respuesta = tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True,
    )
    return respuesta.strip()

# 🔍 Test previo (modelo base, aún sin el PDF)
print("=== Qwen3 BASE ===")
print(chat_es("¿Qué es la gastritis enfisematosa?"))

=== Qwen3 BASE ===
La **gastritis enfisematosa** es un trastorno raro y poco común de la mucosa gástrica en el que se observan **vesículas o burbujas de aire** dentro de la capa mucosa del estómago. No es una enfermedad común ni clínica frecuente, y su diagnóstico suele ser incidental durante una endoscopia.

### Características principales:
- **No es una enfermedad inflamatoria típica** como la gastritis por erosión o por *Helicobacter pylori*.
- Se caracteriza por la presencia de **espacios llenos de aire** (enfisema) en la mucosa gástrica, especialmente en la zona del esófago-gástrico.
- Puede estar asociada a **trastornos de la motilidad gástrica**, como la **disfunción del esfínter esofágico** o **reflujo gastroesofágico crónico**.
- A veces se observa en pacientes con **síndrome de reflujo gastroesofágico (RGE)** o con **trast


In [ ]:
from google.colab import files

print("📂 Sube el PDF del artículo clínico (ej: S2444382416000444.pdf)")
uploaded = files.upload()

PDF_FILENAME = list(uploaded.keys())[0]
print("Usando archivo:", PDF_FILENAME)

📂 Sube el PDF del artículo clínico (ej: S2444382416000444.pdf)


Saving S2444382416000444.pdf to S2444382416000444.pdf
Usando archivo: S2444382416000444.pdf


In [ ]:
from pypdf import PdfReader

reader = PdfReader(PDF_FILENAME)
pages_text = []

for i, page in enumerate(reader.pages):
    txt = page.extract_text()
    if txt:
        pages_text.append(txt)

full_text = "\n".join(pages_text)

print("Caracteres totales del artículo:", len(full_text))
print("Vista previa:\n")
print(full_text[:])

Caracteres totales del artículo: 15462
Vista previa:

Gastroenterol Hepatol. 2016;39(6):393- - -402
www.elsevier.es/gastroenterologia
Gastroenterología y Hepatología
SCIENTIFIC LETTERS
Emphysematous gastritis:
Effectiveness of early antibiotic
therapy/H22845
Gastritis
 enﬁsematosa,  eﬁcacia  del
tratamiento
 con antibioterapia precoz
The
 presence of air in regions of the abdomen may be due to
2
 diseases: gastric emphysema and emphysematous gastri-
tis,
 which differ in aetiology , clinical presentation, evolution
and
 
prognosis.1,2
Gastric
 
emphysema
 was described by Brouardel in 1985
as
 a generally benign disease caused by disruption of the
gastric
 mucosa and subsequent inﬁltration  of the mucosa by
air .
 It is usually a self-limiting disease and resolves without
sequelae.2
Emphysematous gastritis
 
is
 a rare and severe gas-forming
infection
 
in
 the gastric wall. It is caused by the dis-
ruption
 
of
 the mucosa and subsequent invasion of the
gastric
 wall
 
by
 gas-forming

In [ ]:
from datasets import Dataset

def chunk_text(text, max_chars=1024, overlap=200):
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(n, start + max_chars)
        chunk = text[start:end].strip()
        if len(chunk) > 1000:   # evitamos trozos muy cortos
            chunks.append(chunk)
        if end == n:
            break
        start = end - overlap
    return chunks

chunks = chunk_text(full_text, max_chars=1024, overlap=200)
print("Número de fragmentos generados:", len(chunks))

# Mini dataset didáctico (puedes quitar el select para usar todo)
max_samples = min(64, len(chunks))
dataset = Dataset.from_dict({"text": chunks[:max_samples]})

print(dataset)
print("\nEjemplo de fragmento:\n", dataset[0]["text"][:500])

Número de fragmentos generados: 18
Dataset({
    features: ['text'],
    num_rows: 18
})

Ejemplo de fragmento:
 Gastroenterol Hepatol. 2016;39(6):393- - -402
www.elsevier.es/gastroenterologia
Gastroenterología y Hepatología
SCIENTIFIC LETTERS
Emphysematous gastritis:
Effectiveness of early antibiotic
therapy/H22845
Gastritis
 enﬁsematosa,  eﬁcacia  del
tratamiento
 con antibioterapia precoz
The
 presence of air in regions of the abdomen may be due to
2
 diseases: gastric emphysema and emphysematous gastri-
tis,
 which differ in aetiology , clinical presentation, evolution
and
 
prognosis.1,2
Gastric
 
emp


In [ ]:
dataset

In [ ]:
from trl import SFTTrainer, SFTConfig

training_args = SFTConfig(
    output_dir                    = "qwen3_gastritis_finetune",
    per_device_train_batch_size   = 1,
    gradient_accumulation_steps   = 4,
    warmup_steps                  = 5,
    max_steps                     = 100,        # 👈 demo pequeña; súbelo si quieres entrenar más
    logging_steps                 = 5,
    learning_rate                 = 2e-4,
    bf16                          = False, #torch.cuda.is_available()
    max_seq_length                = max_seq_length,
    packing                       = True,      # empaqueta varios textos por secuencia
    save_strategy                 = "no",      # para no llenar disco en el ejemplo
)

trainer = SFTTrainer(
    model            = model,
    tokenizer        = tokenizer,
    train_dataset    = dataset,
    args             = training_args,
    dataset_text_field = "text",  # columna que contiene el texto
)

print("✅ Trainer configurado, listo para entrenar.")

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/18 [00:00<?, ? examples/s]

Unsloth: Packing train dataset (num_proc=6):   0%|          | 0/18 [00:00<?, ? examples/s]

🦥 Unsloth: Packing enabled - training is >2x faster and uses less VRAM!
✅ Trainer configurado, listo para entrenar.


In [ ]:
print("🚀 Empezando fine-tuning suave de Qwen3 con el artículo clínico...")
trainer.train()
print("✅ Fine-tuning terminado (demo).")

The model is already on multiple devices. Skipping the move to device specified in `args`.


🚀 Empezando fine-tuning suave de Qwen3 con el artículo clínico...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 16 | Num Epochs = 25 | Total steps = 100
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 33,030,144 of 4,055,498,240 (0.81% trained)


Step,Training Loss
5,0.060200
10,0.073600
15,0.034400
20,0.042400
25,0.024800
30,0.017100
35,0.010700
40,0.011600
45,0.005600
50,0.004200


train/epoch,▁▁▂▂▂▃▃▄▄▄▅▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇███
train/global_step,▁▁▂▂▂▃▃▄▄▄▅▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇███
train/grad_norm,▃▃▃▄▃▅▅▇▄▇█▂▅▂▄▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▇███████▇▇▇▇█▇▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▁▁
train/loss,█▇▇▆▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,2862102747648000.0
train/epoch,25
train/global_step,100
train/grad_norm,0.0354
train/learning_rate,0.0
train/loss,0.0017


✅ Fine-tuning terminado (demo).


In [ ]:
save_dir = "/content/drive/MyDrive/Qwen3_finetunnig_1000"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print(f"✅ Adaptador LoRA guardado en: {save_dir}")

✅ Adaptador LoRA guardado en: /content/drive/MyDrive/Qwen3_finetunnig_1000


In [ ]:
# Aseguramos modo inferencia tras el entrenamiento
FastLanguageModel.for_inference(model)

pregunta = (
    "Basándote en el artículo de gastritis enfisematosa usado para el fine-tuning, "
    "resume las causas, factores de riesgo y mortalidad que se reportan."
)

print("=== Qwen3 AFINADO CON EL PDF ===")
print(chat_es(pregunta, max_new_tokens=920))

=== Qwen3 AFINADO CON EL PDF ===
Basado en el artículo de gastritis enfisematosa (también conocida como gastritis con enfisema o gastritis enfisematosa), se resumen los siguientes puntos:

**Causas:**  
La gastritis enfisematosa no tiene una causa única definida, pero se asocia principalmente con:

- **Infección por *Helicobacter pylori***: uno de los factores más comunes, aunque no siempre presente.
- **Lesiones crónicas de la mucosa gástrica** por inflamación prolongada.
- **Factores químicos o irritantes** como alcohol, fármacos (especialmente NSAIDs), y tabaco.
- **Alteraciones en la función del sistema digestivo** y disfunción del manto mucoso.

**Factores de riesgo:**  
- Edad avanzada (más común en adultos mayores).  
- Historial de enfermedades gastrointestinales crónicas.  
- Consumo crónico de alcohol o medicamentos antiinflamatorios no esteroideos (NSAIDs).  
- Tabaquismo.  
- Enfermedades autoinmunes o condiciones sistémicas como la enfermedad de Crohn o la enfermedad de Ha